# 실전 문제 해결 (과적합)

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 15

## 모델 정의

### 기존 overfitting model


In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1_1 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')
        self.conv1_2 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv2_1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.conv2_2 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv3_1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3_2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)

        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

### layernormalization과 kernel regularizer( l2 regularization )을 이용한 과적합 해결

In [4]:
class ConvLNReluBlock(tf.keras.Model):
  def __init__(self, filters, kernel_size):
    super(ConvLNReluBlock, self).__init__()
    self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', use_bias = False)
    self.ln = tf.keras.layers.LayerNormalization()

  def call(self, x, training=False, mask=None):
    x = self.conv(x)
    x = self.ln(x)
    return tf.nn.relu(x)

    # layernormalization시 어차피 training와 testing시 같은 연산을 하므로 training을 따로 설정하지 않아도 됌

In [5]:
class MyModel2(tf.keras.Model):
    def __init__(self):
        super(MyModel2, self).__init__()
        self.conv1_1 = ConvLNReluBlock(16, (3, 3))
        self.conv1_2 = ConvLNReluBlock(16, (3, 3))
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv2_1 = ConvLNReluBlock(32, (3, 3))
        self.conv2_2 = ConvLNReluBlock(32, (3, 3))
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv3_1 = ConvLNReluBlock(64, (3, 3))
        self.conv3_2 = ConvLNReluBlock(64, (3, 3))

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))

    def call(self, x, training=False, mask=None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)

        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 데이터셋 준비


In [6]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(1024)

## Keras API 모델 학습

In [8]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/15
1563/1563 [==============================] - 20s 12ms/step - loss: 1.4260 - accuracy: 0.4782 - val_loss: 1.2362 - val_accuracy: 0.5703
Epoch 2/15
1563/1563 [==============================] - 18s 12ms/step - loss: 0.9831 - accuracy: 0.6537 - val_loss: 0.9190 - val_accuracy: 0.6782
Epoch 3/15
1563/1563 [==============================] - 18s 11ms/step - loss: 0.7794 - accuracy: 0.7236 - val_loss: 0.8208 - val_accuracy: 0.7154
Epoch 4/15
1563/1563 [==============================] - 18s 11ms/step - loss: 0.6196 - accuracy: 0.7804 - val_loss: 0.7885 - val_accuracy: 0.7339
Epoch 5/15
1563/1563 [==============================] - 18s 11ms/step - loss: 0.4705 - accuracy: 0.8330 - val_loss: 0.8644 - val_accuracy: 0.7342
Epoch 6/15
1563/1563 [==============================] - 18s 11ms/step - loss: 0.3361 - accuracy: 0.8813 - val_loss: 1.0266 - val_accuracy: 0.7278
Epoch 7/15
1563/1563 [==============================] - 18s 11ms/step - loss: 0.2319 - accuracy: 0.9182 - val_loss: 1.1953 -

In [9]:
model2 = MyModel2()
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model2.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/15
1563/1563 [==============================] - 57s 35ms/step - loss: 2.1577 - accuracy: 0.4260 - val_loss: 1.3409 - val_accuracy: 0.5652
Epoch 2/15
1563/1563 [==============================] - 55s 35ms/step - loss: 1.2641 - accuracy: 0.5965 - val_loss: 1.1598 - val_accuracy: 0.6404
Epoch 3/15
1563/1563 [==============================] - 55s 35ms/step - loss: 1.1067 - accuracy: 0.6524 - val_loss: 1.1281 - val_accuracy: 0.6532
Epoch 4/15
1563/1563 [==============================] - 54s 35ms/step - loss: 1.0028 - accuracy: 0.6908 - val_loss: 1.0032 - val_accuracy: 0.6908
Epoch 5/15
1563/1563 [==============================] - 54s 35ms/step - loss: 0.9285 - accuracy: 0.7162 - val_loss: 0.9391 - val_accuracy: 0.7164
Epoch 6/15
1563/1563 [==============================] - 54s 35ms/step - loss: 0.8713 - accuracy: 0.7356 - val_loss: 0.9087 - val_accuracy: 0.7225
Epoch 7/15
1563/1563 [==============================] - 55s 35ms/step - loss: 0.8247 - accuracy: 0.7533 - val_loss: 0.9026 -